In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 

In [2]:
df=pd.read_excel("customer data.xlsx")
df

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
sns.scatterplot(x=df['Annual Income (k$)'],y=df['Spending Score (1-100)'])
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
df_scale=ss.fit_transform(df)

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
import warnings
warnings.filterwarnings('ignore')
wcss=[]
for i in range(2,11):
    km=KMeans(n_clusters=5)
    km.fit(df_scale)
    #wcss
    wcss.append(km.inertia_)
wcss

In [ ]:
plt.title('Elbow Method for K-Means')
plt.plot(range(2,11),wcss)
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.grid(True)
plt.show()

In [ ]:
df['Kmean_Cluster']=km.predict(df_scale)

In [ ]:
sns.scatterplot(data=df,x='Annual Income (k$)',y='Spending Score (1-100)',hue='Kmean_Cluster',palette='Set1')
plt.show()

In [ ]:
blue   cluster= low income and low spending
red    cluster= high income and high spending
orange cluster= high income and low spending
purple cluster= low income and high spending
green  cluster= median income and median spending


In [ ]:
df[(df['Kmean_Cluster'] == 0) | (df['Kmean_Cluster'] == 4)]

In [ ]:
from scipy.cluster.hierarchy import dendrogram , linkage

In [ ]:
plt.figure(figsize=(10,8))
linked=linkage(df_scale,method='ward')
dendrogram(linked)
plt.title('Dendrogram (Agglomerative Clustering )')
plt.xlabel('Data Points')
plt.ylabel('Distance')
plt.show()

In [ ]:
from sklearn.cluster import AgglomerativeClustering

In [ ]:
ac=AgglomerativeClustering(n_clusters=5)
df['Agglomerative Cluster']=ac.fit_predict(df_scale)

In [ ]:
plt.title('Agglomerative Clustering')
sns.scatterplot(data=df, x='Annual Income (k$)', y='Spending Score (1-100)', hue='Agglomerative Cluster', palette='Set1')
plt.show()

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

In [ ]:
ne=NearestNeighbors(n_neighbors=4)
ne_fit=ne.fit(df_scale)
distances, indices = ne_fit.kneighbors(df_scale)

In [ ]:
distances = np.sort(distances[:, 3])

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(distances)
plt.title('K-distance Graph [DBSCAN]')
plt.xlabel('Points sorted by distance')
plt.ylabel('4th Nearest Neighbor Distance')
plt.grid(True)
plt.show()

In [ ]:
dbscan=DBSCAN(eps=0.4)

In [ ]:
df['DBSCAN Cluster']=dbscan.fit_predict(df_scale)

In [ ]:
plt.title('DBSCAN Clustering')
sns.scatterplot(data=df, x='Annual Income (k$)', y='Spending Score (1-100)', hue='DBSCAN Cluster', palette='Set1')
plt.legend(bbox_to_anchor=(1.05,1))
plt.show()

In [ ]:
Feature               K-Means        Heirarchical
Need to choose k      Yes             Yes
Shape of clusters     Round-ish       Tree-like
Can detect noise      NO              No
Scaling               Yes             Yes
Graph that helped     Elbow method    Dendrogam